## Explore `sdf` output
`sdf` generates a large amount of information during fitting. Most of this is saved in a database that isn't visible on the web, and also in `pickle` files that can be found for each model under the "..." link.

A simpler output is the `json` files under the "model" link, there is less detail here, but they are sufficient for plotting.

To just explore the output you can probably avoid installing the `sdf` package.

In [1]:
import requests
import pickle

### json output
To explore json output we don't need any special packages. Either download directly with `requests`, or open with the `json` module.

In [2]:
r = requests.get('http://drgmk.com/sdb/seds/masters/'
                 'sdb-v2-132436.10-513016.1/public/sdb-v2-132436.10-513016.1-mnest/phoenix_m+modbb_disk_r_.json')

In [3]:
d = r.json()

In [4]:
for k in d.keys():
    print(k, type(d[k]))

id <class 'str'>
write_time <class 'float'>
model_comps <class 'list'>
main_results <class 'list'>
parameters <class 'list'>
best_params <class 'list'>
best_params_1sig <class 'list'>
chisq <class 'float'>
phot_band <class 'list'>
phot_wavelength <class 'list'>
phot_fnujy <class 'list'>
phot_e_fnujy <class 'list'>
phot_upperlim <class 'list'>
phot_ignore <class 'list'>
model_comp_fnujy <class 'list'>
model_comp_fnujy_1sig <class 'list'>
model_total_fnujy <class 'list'>
model_total_fnujy_1sig <class 'list'>
spectra <class 'list'>
model_spectra <class 'list'>
star_spec <class 'dict'>
disk_spec <class 'dict'>


The information contained in the json is largely related to the observational data, e.g. photometry and models in the observed bands.

There are also spectra for each model component.

### pickle output
To explore the pickle data we need the `pickle` package. There is a tonne of information saved here, including fluxes for the models in all bands known to `sdf`.

In [5]:
s = requests.get('http://drgmk.com/sdb/seds/masters/'
                 'sdb-v2-132436.10-513016.1/public/sdb-v2-132436.10-513016.1-mnest/phoenix_m+modbb_disk_r_.pkl')

In [6]:
r = pickle.loads(s.content)

 Config
  found files:
['/Users/grant/Library/CloudStorage/OneDrive-UniversityofWarwick/astro/projects/sdf/sdf/sdf/sdf.conf', '/Users/grant/.sdf.conf']


In [7]:
# print the model component fluxes for the MIRI bands
print(f'filter: {r.model_comps}, total')
for i,f in enumerate(r.all_filters):
    if 'MIRI' in f:
        print(f, r.all_comp_phot[:,i], r.all_phot[i])

filter: ('phoenix_m', 'modbb_disk_r'), total
MIRI.F560W [0.22747165 0.00036146] 0.22868825959040914
MIRI.F770W [0.12863864 0.00108924] 0.1303280009095618
MIRI.F1000W [0.07618979 0.00189427] 0.07839527048226816
MIRI.F1280W [0.05867608 0.00220988] 0.061098058769063206
MIRI.F1130W [0.04654477 0.00242885] 0.04907121744937672
MIRI.F1500W [0.03386959 0.00255847] 0.03645797741836954
MIRI.F1800W [0.02370804 0.00248764] 0.026191606212937323
MIRI.F2100W [0.01791005 0.00231372] 0.020245121862749044
MIRI.F2550W [0.0119938  0.00204449] 0.014038587641430187
